####Variant Filtration pipeline for FVL modifiers project

##Fastq files

All generated fastq files have been deposited to the NCBI Sequence Read Archive (project accession number #PRJNA397141)

#PRJNA397141 encompasses 108 whole exome sequencing experiments from 108 different mice

Current pipeline calls variants jointly from 107 sequencing experiments.
1 sample (G2 offspring of pedigree 1) is excluded from the joint analysis because G1 of the same pedigree is already represented in the cohort.

##Overview of variant calling pipeline

Reference genome: Mus_musculus GRCm38 release 73, downloaded from Ensemble

**Used software**: 
* bwa (version 0.7.5a-r405)
* picard-tools (version 1.105)
* GenomeAnalysisTK (version 2.6.5)

**Steps to obtain the .vcf files:**
1. bwa aln -q 15
 bwa sampe
2. picard-tools/SortSam.jar SORT_ORDER=coordinate VALIDATION_STRINGENCY=SILENT CREATE_INDEX=true
3. picard-tools/MarkDuplicates.jar VALIDATION_STRINGENCY=SILENT REMOVE_DUPLICATES=true ASSUME_SORTED=true CREATE_INDEX=true
4. GenomeAnalysisTK/GenomeAnalysisTK.jar -T RealignerTargetCreator
5. GenomeAnalysisTK/GenomeAnalysisTK.jar -T IndelRealigner
6. picard-tools/FixMateInformation.jar SORT_ORDER=coordinate VALIDATION_STRINGENCY=SILENT CREATE_INDEX=true
7. GenomeAnalysisTK/GenomeAnalysisTK.jar -T HaplotypeCaller -stand_call_conf 50.0 stand_emit_conf
8. GenomeAnalysisTK/GenomeAnalysisTK.jar -T VariantAnnotator -A VariantType
9. GenomeAnalysisTK/GenomeAnalysisTK.jar -T SelectVariants -selectType SNP 
10. GenomeAnalysisTK/GenomeAnalysisTK.jar -T SelectVariants -selectType INDEL
11. GenomeAnalysisTK/GenomeAnalysisTK.jar -T VariantFiltration --variant SNP.vcf --filterExpression 'QD < 2.0 || MQ < 40.0 || FS > 60.0 || HaplotypeScore > 13.0 || MQRankSum < -12.5 || ReadPosRankSum < -8.0' --filterName 'FAIL'
12. GenomeAnalysisTK/GenomeAnalysisTK.jar -T VariantFiltration --variant INDEL.vcf --filterExpression "QD < 2.0 || FS > 200.0 || ReadPosRankSum < -20.0 || InbreedingCoeff < -0.8" --filterName FAILED

_End result is two vcf files: SNP.vcf and INDEL.vcf_

##Annotation of called SNP.vcf and INDEL.vcf

Details on Annovar version and reference sequence:

1. $LastChangedDate: 2012-05-25
2. annotate_variation.pl --buildver mm10 --downdb seq mousedb/mm10_seq
_Downloading annotation database ftp://hgdownload.cse.ucsc.edu/goldenPath/mm10/bigZips/chromFa.tar.gz ... OK_

In [ ]:
#UNIX commands:

cat *.vcf > catted.vcf
sort -k1n -k2n catted.vcf > all.vcf
grep '#CHROM' all.vcf | head -1 > header.txt

convert2annovar.pl all.vcf -format vcf4old -includeinfo > all_annovar
annotate_variation.pl all_annovar --buildver mm10 /database/annovar/mousedb/

##Preparation of files for filtration

###Merge and reformat Annovar output files back into one .vcf

Input files are Annovar output files 'all_annovar.exonic_variant_function' and 'all_annovar.variant_function'

filtering the **annotate_vcf.py** script merges Annovar annotation files 'all_annovar.exonic_variant_function' and 'all_annovar.variant_function' back into one **all_annovar_annot.vcf** file, adding Annovar annotation to the INFO column

In [ ]:
##UNIX command:
./annotate_vcf.py all_annovar

**annotate_vcf.py**:

In [ ]:
#!/usr/bin/env python
#This script merges Annovar annotation files back into .vcf format putting annotation information under the INFO column

import sys
import re

FileName=sys.argv[1]

#read in annovar input files
input1=open(FileName+'.variant_function','r')
input2=open(FileName+'.exonic_variant_function','r')
#read in the header for .vcf
header=open('header.txt','r').readline().rstrip()
#define the output file
outfile=open(FileName+'_annot.vcf','w')

rows=[]
exonic=dict()

print >> outfile, header

#annotate the exonic variants
for line in input2:
    line=line.strip()
    row=line.split('\t')
    row[15]=row[15]+';AnnovarType='+'_'.join(row[1].split())+';AnnovarGene='+'_'.join(row[2].split())
    new_line=row[8:len(row)]        
    new_line='\t'.join(new_line)
    exonic[(row[3],row[4])]=new_line

#read in all the variants
for line in input1:
    line=line.strip()
    row=line.split('\t')

    #if variant already annotated in exonic, print and skip the rest
    if (row[2],row[3]) in exonic:
        print >> outfile, exonic[(row[2],row[3])]
        continue
    #otherwise annotate with available info
    else:
        row[14]=row[14]+';AnnovarType='+'_'.join(row[0].split())+';AnnovarGene='+'_'.join(row[1].split())
        new_line=row[7:len(row)]
        print >> outfile, '\t'.join(new_line)
        continue

input1.close()
input2.close()
outfile.close()


Before filtering, I will remove all the non-exonic variants from the input file using script **exonic_variants.py**:

In [ ]:
#!/usr/bin/env python
#This script will extract variants called within exonic regions

import sys
import re
 
FileName=sys.argv[1]
   
input=open(FileName+'_annovar_annot.vcf','r')
outfile=open(FileName+'_exonic.vcf','w')


for line in input:
	if line.startswith('#'):
		print >> outfile, line.rstrip()

	line=line.rstrip()
	row=line.split('\t')
	for i in row[7].split(';'):
		if i.startswith('AnnovarType='):
			type=i.split('=')[1]
			if (type=='stopgain') or (type=='stoploss') or (type=='synonymous_SNV') or (type=='nonsynonymous_SNV') or (type=='UTR3') or (type=='UTR5') or (type=='splicing') or (type=='frameshift_insertion') or (type=='nonframeshift_insertion') or (type=='frameshift_deletion') or (type=='nonframeshift_deletion') or (type=='ncRNA_splicing') or (type=='ncRNA_exonic'):
				print >> outfile, line
			else:
				break

input.close()
outfile.close()			

After annotation the file is ready for filtering.

First filter removes variants that did not pass GATK filtering
python script is called **GATK.py**

In [ ]:
##UNIX command:
./GATK.py all_exonic

**GATK.py**:

In [ ]:
#!/usr/bin/env python
#This script removes all variants that did not pass GATK filters

import sys
import re
import vcf

FileName=sys.argv[1]

vcfFile = vcf.Reader( open(FileName+'.vcf','r') )
outfile = vcf.Writer(open(FileName+'_filter1.vcf', 'w'), vcfFile)
filter1 = vcf.Writer(open('filter1_removed.vcf', 'w'), vcfFile)

#filters statistics
counter=0
f1=0
passed=0

for record in vcfFile:  
    counter+=1    
   
    #remove variants that did not pass GATK filters
    if len(record.FILTER)>0:
        f1+=1
        filter1.write_record(record)
        continue
        
    passed+=1
    outfile.write_record(record)

outfile.close()
filter1.close()

Second filter **unique.py** will remove all variants not uniquely present in only one mouse.

In [ ]:
##UNIX command:
./unique.py all_exonic

**unique.py**:

In [ ]:
#!/usr/bin/env python
#This script removes all variants that are called as hets in more than one mouse, or called as homozygous non-ref

import sys
import re
import vcf

FileName=sys.argv[1]

vcfFile = vcf.Reader( open(FileName+'_filter1.vcf','r') )
outfile = vcf.Writer(open(FileName+'_filter2.vcf', 'w'), vcfFile)
filter2 = vcf.Writer(open('filter2_removed.vcf', 'w'), vcfFile)

#filters statistics
counter=0
f2=0
passed=0

for record in vcfFile:  
    counter+=1    
   
    #remove variants where other mice shared the same variant allele as het or alt-hom    
    if record.num_het>1 or record.num_hom_alt>0:
        f2+=1
        filter2.write_record(record)
        continue
        
    passed+=1
    outfile.write_record(record)

outfile.close()
filter2.close()

Third filter will remove variants called within the same gene more than once within any given mouse using **unique_genes.py**.


In [ ]:
##UNIX command:
./unique_genes.py all_exonic

**unique_genes.py**:

In [ ]:
#!/usr/bin/env python
#This script removes all variants that are called more than once in any given gene within one mouse

import sys
import re

FileName=sys.argv[1]

input=open(FileName+'_filter2.vcf','r')
outfile=open(FileName+'_filter3.vcf', 'w')

#filters statistics
seq=range(0,107)
genes=dict()

for line in input:
	if line.startswith('#CHROM'):
		line=line.rstrip()
		print >> outfile, line
		row=line.split('\t')
		mouseIDs=row[9:len(row)]
	else:
		line=line.rstrip()
		row=line.split('\t')
		info=row[7].split(';')
		for i in info:
			if i.startswith('AnnovarGene='):
				i=i.split('=')[1]
				gene=re.split('[( :]',i)[0]
		data=row[9:len(row)]
		for i in seq:
			if data[i].startswith('0/1'):
				if (mouseIDs[i],gene) in genes:
					genes[(mouseIDs[i],gene)]='multiple'
				else:
					genes[(mouseIDs[i],gene)]=line

for gene in genes:
	if genes[gene]=='multiple':
		print gene
		continue
	else:
		print >> outfile, genes[gene]

input.close()
outfile.close()

Forth filter **RD.py** will remove variants with less than 6 independent reads and an allelic ratio more than 3 fold different

In [ ]:
 ##UNIX command:
./RD.py all_exonic

**RD.py**:

In [ ]:
#!/usr/bin/env python
#This script removes all variants that not covered with RD 6, and ratio 1:3
import sys
import re

FileName=sys.argv[1]

input=open(FileName+'_filter3.vcf','r')
outfile=open(FileName+'_filter4.vcf', 'w')

for line in input:
	if line.startswith('#CHROM'):
		line=line.rstrip()
		print >> outfile, line

	else:
		mouse=''
		line=line.rstrip()
		row=line.split('\t')
		info=row[7].split(';')
		data=row[9:len(row)]
		for mouse in data:
			if mouse.startswith('0/1'):
				het=mouse.split(':')
		RD=het[2]
		if not RD.isdigit():
			continue		
		#remove calls with less than 6 reads
		if int(RD) < 6:
			continue
		REF=int(het[1].split(',')[0])
		ALT=int(het[1].split(',')[1])
		#remove calls where Ref or Alt alleles have count 0
		if REF==0 or ALT==0:
			continue
		#remove reads where Ref/Alt ratio is off more than 3 fold
		dev=float(REF)/float(ALT)
		if dev <0.3 or dev > 3:
			continue

		print >> outfile, line

input.close()
outfile.close()

**ADDITIONAL FLTERING**

At this point (file: all_exonic_filter4.vcf) the variant filtering is finished and the data is used for ENU mutation analysis. Additional filtering steps below are to prepare the burden analysis of ENU protein altering variants.

We extract protein coding variants using script **coding.py**

In [ ]:
##UNIX command:
./RD.py all_exonic

**coding.py**:

In [ ]:
#!/usr/bin/env python
#This script will extract variants called within protein coding regions

import sys
import re
 
FileName=sys.argv[1]
   
input=open(FileName+'_filter4.vcf','r')
outfile=open(FileName+'_filter5.vcf','w')


for line in input:
	if line.startswith('#'):
		print >> outfile, line.rstrip()

	line=line.rstrip()
	row=line.split('\t')
	for i in row[7].split(';'):
		if i.startswith('AnnovarType='):
			type=i.split('=')[1]
			if (type=='stopgain') or (type=='stoploss') or (type=='nonsynonymous_SNV') or (type=='splicing') or (type=='frameshift_insertion') or (type=='frameshift_deletion'):
				print >> outfile, line
			else:
				break

input.close()
outfile.close()	

Next step will count how many variants per each gene are in the data set and print out a tab-deliminated text with selected information from the .vcf file. This is done by using **./count_genes.py**.

In [ ]:
##UNIX command:
./count_genes.py all_exonic

**count_genes.py**:

In [ ]:
#!/usr/bin/env python
#This script counts the number of variants and prints out a tab-deliminated text file of main information

import sys
import re

FileName=sys.argv[1]

input=open(FileName+'_filter5.vcf','r')
outfile=open(FileName+'_final.txt', 'w')

#filters statistics
seq=range(0,107)
genes=dict()

for line in input:
	if line.startswith('#CHROM'):
		continue
	else:
		line=line.rstrip()
		row=line.split('\t')
		info=row[7].split(';')
		for i in info:
			if i.startswith('AnnovarGene='):
				all_gene=i.split('=')[1]
				gene=re.split('[( :]',all_gene)[0]
				if gene in genes:
					genes[gene]+=1
				else:
					genes[gene]=1

input.close()

input=open(FileName+'_filter5.vcf','r')
print >> outfile, 'CHROM\tPOS\tREF\tALT\tQUAL\tAnnovarType\tAnnovarGene\tData\tmouseID\tGene\tCount'

for line in input:
	if line.startswith('#CHROM'):
		line=line.rstrip()
		row=line.split('\t')
		mouseIDs=row[9:len(row)]
	else:
		line=line.rstrip()
		row=line.split('\t')
		info=row[7].split(';')
		for i in info:
			if i.startswith('AnnovarGene='):
				all_gene=i.split('=')[1]
				gene=re.split('[( :]',all_gene)[0]
			elif i.startswith('AnnovarType='):
				type=i.split('=')[1]
		alldata=row[9:len(row)]
		for i in seq:
			if alldata[i].startswith('0/1'):
				data=alldata[i]
				mouseID=mouseIDs[i]

		print >> outfile, row[0]+'\t'+row[1]+'\t'+row[3]+'\t'+row[4]+'\t'+row[5]+'\t'+type+'\t'+all_gene+'\t'+data+'\t'+mouseID+'\t'+gene+'\t'+str(genes[gene])

input.close()
outfile.close()

Next simulation p-values are added to the dataset using **add_pval.py**. Look at **FVLmod_Simulation.ipynb** for details of how p-values were identified. 

In [ ]:
##UNIX command:
./add_pval.py all_exonic

**add_pval.py**:

In [ ]:
#!/usr/bin/env python
#add the pvalue from simulations to the final variant file

import sys
import re

#create filenames
fileName=sys.argv[1]
pvalues=sys.argv[2]

input=open(fileName+'.txt','r')
outfile=open(fileName+'_pval.txt','w')
pval=open(pvalues,'r')

genes=dict()

for line in pval:
	line=line.strip()
	row=line.split('\t')
	genes[row[1]]=line

for line in input:
	if line.startswith('CHROM'):
		print >> outfile, line.strip()+'\tpval'
		continue
	line=line.strip()
	row=line.split('\t')
	row[0]
	if row[0]=='X':
		row[0]='20'
	gene=row[9]
	AA=int(row[10])
	if gene in genes:
		data=genes[gene].split('\t')
		p=float(data[5+AA])/10000000
		print >> outfile, '\t'.join(row)+'\t'+str(p)
	else:
		print 'gene not in genes'
		print >> outfile, '\t'.join(row)+'\tNA'